In [1]:
# Packages for data manipulation
import pandas as pd
import numpy as np

# Recommender package found at https://github.com/ZWMiller/svdRec
from svdRec import svdRec

In [2]:
# Importing Steam data from 2016 into a pandas dataframe

df = pd.read_csv('./steam_data.csv')

In [3]:
df.head()

,User ID,Game Title,Hours Played
0,151603712,The Elder Scrolls V Skyrim,273.0
1,151603712,Fallout 4,87.0
2,151603712,Spore,14.9
3,151603712,Fallout New Vegas,12.1
4,151603712,Left 4 Dead 2,8.9


In [4]:
# Creating Game ID column using pandas .ngroup()

df['Game ID'] = df.groupby(['Game Title']).ngroup()

In [5]:
# Exporting to csv

df.to_csv('steam_data_w_game_id.csv')

In [6]:
df.head()

,User ID,Game Title,Hours Played,Game ID
0,151603712,The Elder Scrolls V Skyrim,273.0,3067
1,151603712,Fallout 4,87.0,1162
2,151603712,Spore,14.9,2813
3,151603712,Fallout New Vegas,12.1,1163
4,151603712,Left 4 Dead 2,8.9,1733


In [27]:
# Creating a new dataframe to pass into svdRec

df2 = pd.DataFrame(df, columns=['User ID','Game ID', 'Hours Played'])

In [28]:
df2.head()

,User ID,Game ID,Hours Played
0,151603712,3067,273.0
1,151603712,1162,87.0
2,151603712,2813,14.9
3,151603712,1163,12.1
4,151603712,1733,8.9


In [29]:
# Checking for null values

df2.isnull().sum()

User ID         0
Game ID         0
Hours Played    0
dtype: int64

In [31]:
# .info() to check data types

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70489 entries, 0 to 70488
Data columns (total 3 columns):
User ID         70489 non-null int64
Game ID         70489 non-null int64
Hours Played    70489 non-null float64
dtypes: float64(1), int64(2)
memory usage: 1.6 MB


In [38]:
# Verifying index had no negative column index (prior error received)

for cols in df2.columns.tolist()[1:]:
    df2 = df2.loc[df2[cols] > 0]

In [36]:
# Exporting dataframe to pass into svdRec

df2.to_csv('steam_data_wo_game_title.csv', index=False)

In [37]:
# Passing csv title into svdRec

svd = svdRec.svdRec()
svd.load_csv_sparse('steam_data_wo_game_title.csv', delimiter=',', skiprows=1)

Note: load_csv_sparse expects a csv in the format of: rowID, colID, Value, ...
Created matrix of shape:  (309903146, 3599)


In [48]:
# Passing csv title into svdRec

svd.SVD(num_dim=1)

In [50]:
# Creating a dictionary of game ID's to game titles and passing it to the load_item_encoder

steam_dict = {} 
for i, row in df.iterrows():
    steam_dict.update({row['Game ID']: row['Game Title']})
svd.load_item_encoder(steam_dict)

In [51]:
# Commented out the steam_dict due to the wall of text

# Steam_dict

{3067: 'The Elder Scrolls V Skyrim',
 1162: 'Fallout 4',
 2813: 'Spore',
 1163: 'Fallout New Vegas',
 1733: 'Left 4 Dead 2',
 1535: 'HuniePop',
 2197: 'Path of Exile',
 2251: 'Poly Bridge',
 1732: 'Left 4 Dead',
 2994: 'Team Fortress 2',
 3247: 'Tomb Raider',
 3024: 'The Banner Saga',
 783: 'Dead Island Epidemic',
 348: 'BioShock Infinite',
 934: 'Dragon Age Origins - Ultimate Edition',
 1161: 'Fallout 3 - Game of the Year Edition',
 2535: 'SEGA Genesis & Mega Drive Classics',
 1372: 'Grand Theft Auto IV',
 2401: 'Realm of the Mad God',
 1863: 'Marvel Heroes 2015',
 1025: 'Eldevin',
 922: 'Dota 2',
 346: 'BioShock',
 2482: 'Robocraft',
 1313: "Garry's Mod",
 1619: 'Jazzpunk',
 3327: 'Ultra Street Fighter IV',
 1127: 'FINAL FANTASY XIII',
 2691: "Sid Meier's Civilization V",
 1700: 'L.A. Noire',
 641: 'Company of Heroes Tales of Valor',
 27: '7 Days to Die',
 895: 'Divekick',
 1124: 'FINAL FANTASY VII',
 2132: 'Orcs Must Die! 2',
 1657: 'Killing Floor',
 636: 'Company of Heroes',
 292: 

In [105]:
# Testing out game recommendation for Dota 2. Even though the code errors out, the results are still provided via the game id.
# I manually looked up each code in the steam_dict to obtain the name of the games.
# The top 4 games using this recommender were CS:GO, Counter Strike, Team Fortress 2 and Conunter Strike Source.
# Thus, the recommender solely recommended popular games, not games of a similar genre. Thus, I did not
# go any further with this recommender system.


exceptions = []

GAME_ID = 922 # 922 is Dota 2
for item in steam_dict:
    try:
        print(svd.get_similar_items(GAME_ID,show_similarity=True))
    except:
        print("Game ID exception")
        exceptions.append([GAME_ID])
        break
        
# Counter Strike Global Offensive is game 673
# Counter Strike is game 670
# Team Fortress 2 is game 2994
# Counter Strike source is game 675

[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.0095504476285

[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.0095504476285

[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.0095504476285

[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.0095504476285

[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.0095504476285

...

[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.0095504476285

[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.0095504476285

[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.0095504476285

[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.0095504476285

[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.009550447628555751)]
[(922, 0.9961300421471397), (673, 0.05670672662782649), (670, 0.016146301333866526), (2994, 0.012786768602777608), (675, 0.0095504476285